In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re


pip install webdriver-manager
pip install selenium

In [22]:
# Parkrun blocked my request after a few tries, so I had to use a different approach to get the HTML content
print(soup.prettify())

<html>
 <head>
  <title>
   403 Forbidden
  </title>
 </head>
 <body>
  <center>
   <h1>
    403 Forbidden
   </h1>
  </center>
 </body>
</html>



In [ ]:
# parkrun blocks requests after a few tries, so I had to use a different approach to get the HTML content

import requests
from bs4 import BeautifulSoup
import pandas as pd

# Set headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0 Safari/537.36"
}

# Parkrun results URL (latest Exeter results)
url = "https://www.parkrun.org.uk/exeterriverside/results/latestresults/"
response = requests.get(url, headers=headers)

# Check if request was successful
if response.status_code != 200:
    print(f"Failed to fetch page: {response.status_code}")
else:
    soup = BeautifulSoup(response.text, "html.parser")
    print(soup.prettify()[:1000])  # Print the first 1000 characters of the page
    table = soup.find("table", id="results")
    rows = table.find_all("tr")[1:]  # skip header

    data = []

    for row in rows:
        cols = row.find_all("td")
        if not cols or len(cols) < 6:
            continue

        age_group = cols[5].text.strip()
        # Match common female 18–24 age groups
        if age_group in ["JW19-24", "SW18-19", "F18-24", "SW20-24"]:
            name = cols[1].text.strip()
            time = cols[3].text.strip()
            data.append({
                "Name": name,
                "Time": time,
                "Age Group": age_group
            })

    df = pd.DataFrame(data)
    print(df.head())



<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="/wp-content/themes/parkrun/favicons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/wp-content/themes/parkrun/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/wp-content/themes/parkrun/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/wp-content/themes/parkrun/favicons/site.webmanifest" rel="manifest"/>
  <link color="#2b233d" href="/wp-content/themes/parkrun/favicons/safari-pinned-tab.svg" rel="mask-icon"/>
  <link href="/wp-content/themes/parkrun/favicons/favicon.ico" rel="shortcut icon"/>
  <meta content="#da532c" name="msapplication-TileColor"/>
  <meta content="/wp-content/themes/parkrun/favicons/browserconfig.xml" name="msapplication-config"/>
  <meta content="#ffffff" name="theme-color"/>
  <meta content="Exeter Quay &am


AttributeError: 'NoneType' object has no attribute 'find_all'

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Setup Chrome WebDriver automatically using webdriver_manager, ignore popup
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time

# Setup Chrome options to run in headless mode (no UI)
chrome_options = Options()
chrome_options.add_argument("--headless")  # Runs Chrome in headless mode
chrome_options.add_argument("--no-sandbox")  # Disables the sandbox (useful for certain environments)
chrome_options.add_argument("--disable-dev-shm-usage")  # Useful for containers like Docker

# Setup Chrome WebDriver using webdriver_manager
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Open the park run results page for Exeter Riverside
url = "https://www.parkrun.org.uk/exeterriverside/results/latestresults/"
driver.get(url)

# Allow page to load
time.sleep(5)


In [ ]:
# Bypassing the 403 Forbidden error using Selenium and chrome WebDriver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time

# Set up Chrome options
options = Options()
options.add_argument("--headless")  # So that it runs without a popup window
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

# Launch browser
driver = webdriver.Chrome(options=options)
driver.get("https://www.parkrun.org.uk/exeterriverside/results/latestresults/")

# Allow time for JS to load 
time.sleep(5)

try:
    # Get page source and check for real content
    soup = BeautifulSoup(driver.page_source, "html.parser")

    if "403 Forbidden" in soup.text:
        print("Still blocked: Server returned 403 Forbidden.")
    else:
        print("Page loaded successfully. Proceeding...")
        print(soup.prettify()[:1000])  # show a preview of the real content
finally:
    driver.quit()



Page loaded successfully. Proceeding...
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="/wp-content/themes/parkrun/favicons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/wp-content/themes/parkrun/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/wp-content/themes/parkrun/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/wp-content/themes/parkrun/favicons/site.webmanifest" rel="manifest"/>
  <link color="#2b233d" href="/wp-content/themes/parkrun/favicons/safari-pinned-tab.svg" rel="mask-icon"/>
  <link href="/wp-content/themes/parkrun/favicons/favicon.ico" rel="shortcut icon"/>
  <meta content="#da532c" name="msapplication-TileColor"/>
  <meta content="/wp-content/themes/parkrun/favicons/browserconfig.xml" name="msapplication-config"/>
  <meta content="#ffffff" name="theme-color"/>
  <meta co

In [ ]:
# Had problems finding the table in html, so I printed the soup to find it
tables = soup.find_all("table")

for i, table in enumerate(tables):
    print(f"\n--- Table {i} ---")
    print(table.prettify()[:500])  # Print the first bit of each table


--- Table 0 ---
<table class="Results-table Results-table--compact js-ResultsTable">
 <thead>
  <tr class="Results-table-thead">
   <th class="Results-table-th Results-table-th--position">
    Position
   </th>
   <th class="Results-table-th Results-table-th--name">
    parkrunner
   </th>
   <th class="Results-table-th Results-table-th--gender">
    Gender
   </th>
   <th class="Results-table-th Results-table-th--ageGroup">
    Age Group
   </th>
   <th class="Results-table-th Results-table-th--club">
    Club


In [ ]:
# After inspecting the soup, I found the table I needed

# Set up Chrome options
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

# Launch browser
driver = webdriver.Chrome(options=options)
driver.get("https://www.parkrun.org.uk/exeterriverside/results/latestresults/")
time.sleep(5)

# Get the page source
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# Locate the results table
table = soup.find("table", class_="Results-table")
rows = table.find_all("tr")[1:]  # skip header

data = []

for row in rows:
    cols = row.find_all("td")
    if len(cols) >= 6:
        position = cols[0].text.strip()
        name = cols[1].text.strip()
        gender = cols[2].text.strip()
        age_group = cols[3].text.strip()
        club = cols[4].text.strip()
        time_run = cols[5].text.strip()

        data.append({
            "Position": position,
            "Name": name,
            "Gender": gender,
            "Age Group": age_group,
            "Club": club,
            "Time": time_run
        })

# Convert to DataFrame
df = pd.DataFrame(data)
print(df.head(10))  # Show first 10 results to confirm, this was successful after some debugging


  Position                                               Name  \
0        1  Henry SADDLER-IRVINE117 parkruns | \n         ...   
1        2  Oscar STAPLETON153 parkruns | \n              ...   
2        3  Walter ROSENWALD19 parkruns | \n              ...   
3        4  Ieuan GALVIN101 parkruns | \n                 ...   
4        5  Tim PRATT245 parkruns | \n                    ...   
5        6  Tim LANDY28 parkruns | \n                     ...   
6        7  Will SACHS2 parkruns | \n                     ...   
7        8  Lewys RYAN37 parkruns | \n                    ...   
8        9  Abigail WHEELHOUSE39 parkruns | \n            ...   
9       10  Harriet MITCHELL5 parkruns | \n               ...   

                                Gender                Age Group  \
0    Male\n                      1/183  SM30-3478.54% age grade   
1    Male\n                      2/183  SM20-2476.86% age grade   
2    Male\n                      3/183  SM25-2974.28% age grade   
3    Male\n     

In [31]:
df.head()

,Position,Name,Gender,Age Group,Club,Time
0,1,Henry SADDLER-IRVINE117 parkruns | \n ...,Male\n 1/183,SM30-3478.54% age grade,Dawlish Coasters,16:37PB 15:42
1,2,Oscar STAPLETON153 parkruns | \n ...,Male\n 2/183,SM20-2476.86% age grade,Ilkley Harriers AC,16:47New PB!
2,3,Walter ROSENWALD19 parkruns | \n ...,Male\n 3/183,SM25-2974.28% age grade,,17:22New PB!
3,4,Ieuan GALVIN101 parkruns | \n ...,Male\n 4/183,SM20-2473.57% age grade,Exeter tri club,17:32New PB!
4,5,Tim PRATT245 parkruns | \n ...,Male\n 5/183,SM30-3470.87% age grade,Teignbridge Trotters,18:15PB 16:57


In [43]:
import re

# Clean the "Age Group" column to remove extra percentages and text
df["Age Group"] = df["Age Group"].apply(lambda x: re.sub(r'-\d+(\.\d+)?%.*', '', x))

# Ensure the "Time" column only contains the first 5 characters (xx:xx format)
df["Time"] = df["Time"].apply(lambda x: x[:5] if len(x) >= 5 else x)

# Verify the cleaned columns
print(df[["Age Group", "Time"]].head())


  Age Group   Time
0      SM30  16:37
1      SM20  16:47
2      SM25  17:22
3      SM20  17:32
4      SM30  18:15


In [45]:
# Filter for female age groups relevant to 20-year-old - SW20 really means SW20-24 but regex cleaning above removed the 24
target_age_groups = ["SW20"]

# Filter the DataFrame
filtered_df = df[df["Age Group"].isin(target_age_groups)]

# Reset index and display the result
filtered_df.reset_index(drop=True, inplace=True)
filtered_df.head()

#filtered_df.to_csv("SW20-24_exeter_results.csv", index=False)  # Save to CSV for further analysis  

,Position,Name,Gender,Age Group,Club,Time
0,9,Abigail WHEELHOUSE39 parkruns | \n ...,Female\n 1/165,SW20,Solihull & Small Heath AC,18:53
1,45,Gina DEMONTIS265 parkruns | \n ...,Female\n 6/165,SW20,Egdon Heath Harriers,21:44
2,110,Eloise BEAUMONT4 parkruns | \n ...,Female\n 17/165,SW20,,25:17
3,111,Leoni BARNES2 parkruns | \n ...,Female\n 18/165,SW20,,25:17
4,141,Eloise KHEMLYANI51 parkruns | \n ...,Female\n 27/165,SW20,,26:03
